In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [5]:
from langchain.tools import tool

@tool
def get_weather(location:str):
    """Call to get the current weather"""
    if location.lower() in ["dhaka"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."
    

@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "dhaka" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."
    

tools = [get_weather, check_seating_availability]

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
result = llm_with_tools.invoke("How will be the weather in dhaka today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_en2T75YtZ0XWSvIqEWnR4bp1', 'function': {'arguments': '{"location":"Dhaka"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C09LmlR6wvrDTD3C0AynpXDfT9v13', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--53b63c02-442c-4ff2-ac02-4178e2b1fde7-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Dhaka'}, 'id': 'call_en2T75YtZ0XWSvIqEWnR4bp1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100, 'input_token_details': {'audio': 0, 

In [8]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Dhaka'},
  'id': 'call_en2T75YtZ0XWSvIqEWnR4bp1',
  'type': 'tool_call'}]

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will be the weather today in dhaka? And Will we still have seats outdoor available?"
    )
]

In [12]:
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [13]:
llm_output.tool_calls


[{'name': 'get_weather',
  'args': {'location': 'Dhaka'},
  'id': 'call_SDFpovMtu3KN4xMMqMMob1O7',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Dhaka', 'seating_type': 'outdoor'},
  'id': 'call_zRXBIL8CoXGoa3kL5ZeoTLqF',
  'type': 'tool_call'}]

In [14]:
messages

[HumanMessage(content='How will be the weather today in dhaka? And Will we still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SDFpovMtu3KN4xMMqMMob1O7', 'function': {'arguments': '{"location": "Dhaka"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_zRXBIL8CoXGoa3kL5ZeoTLqF', 'function': {'arguments': '{"location": "Dhaka", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 93, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C09PEhjRNzHW7xj1xYEGh46qmcX4T', 'service_tier': 'default', 'fini

In [15]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [16]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [17]:
messages

[HumanMessage(content='How will be the weather today in dhaka? And Will we still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SDFpovMtu3KN4xMMqMMob1O7', 'function': {'arguments': '{"location": "Dhaka"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_zRXBIL8CoXGoa3kL5ZeoTLqF', 'function': {'arguments': '{"location": "Dhaka", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 93, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C09PEhjRNzHW7xj1xYEGh46qmcX4T', 'service_tier': 'default', 'fini

In [18]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Dhaka today is 15 degrees Celsius and cloudy. There are still seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 180, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C09R6jqOD3BT35sy1yMrz6WjVqhOh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c0b3760c-f1fb-41eb-8d38-0f11fa6756c6-0', usage_metadata={'input_tokens': 180, 'output_tokens': 22, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})